In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
#KAFKA_TOPIC_NAME_CONS = "sdwanalarms"
KAFKA_TOPIC_NAME_CONS = "prod_analytics_jll_tca_job_status"
#KAFKA_OUTPUT_TOPIC_NAME_CONS = "enrichedalarms"
#KAFKA_BOOTSTRAP_SERVERS_CONS = '96.119.88.27:9092'
KAFKA_BOOTSTRAP_SERVERS_CONS ='sdwan-kafka-as-148209.pie.comcast.net:9093,sdwan-kafka-ch2-085864.pie.comcast.net:9093,sdwan-kafka-ho-082563.pie.comcast.net:9093'

if __name__ == "__main__":
  print("PySpark Structured Streaming with Kafka Demo Application Started ...")

  spark = SparkSession \
  .builder \
  .appName("PySpark Structured Streaming with Kafka Demo") \
  .master('local') \
  .config("spark.executor.memory", "7g") \
  .config("spark.driver.memory", "5g") \
  .config("spark.memory.offHeap.enabled", True) \
  .config("spark.memory.offHeap.size", "5g") \
  .getOrCreate()

  spark.sparkContext.setLogLevel("ERROR")
  transaction_detail_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
  .option("kafka.ssl.truststore.location", "dbfs:/FileStore/shared_uploads/Anil_Mohan@cable.comcast.com/client_truststore.jks") \
  .option("kafka.ssl.keystore.location", "dbfs:/FileStore/shared_uploads/Anil_Mohan@cable.comcast.com/client.keystore.jks") \
  .option("kafka.ssl.keystore.password", "changeit") \
  .option("kafka.ssl.truststore.password", "changeit")\
  .option("subscribe", KAFKA_TOPIC_NAME_CONS) \
  .option("startingOffsets", "earliest") \
  .load()

  print("Printing Schema of transaction_detail_df: ")
  transaction_detail_df.printSchema()

  #transaction_detail_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

  stream_df_query = transaction_detail_df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("console") \
  .start()

  stream_df_query.awaitTermination()

PySpark Structured Streaming with Kafka Demo Application Started ...
Printing Schema of transaction_detail_df: 
root
-- key: binary (nullable = true)
-- value: binary (nullable = true)
-- topic: string (nullable = true)
-- partition: integer (nullable = true)
-- offset: long (nullable = true)
-- timestamp: timestamp (nullable = true)
-- timestampType: integer (nullable = true)